## 门控循环单元

当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。虽然裁剪梯度可以应对梯度爆炸，但无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中较难捕捉时间序列中时间步距离较大的依赖关系。

门控循环神经网络（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。它通过可以学习的门来控制信息的流动。其中，**门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络**。

### 1 模型结构

GRU引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式。

![avatr](../resource/gru_1.svg)

重置门和更新门的输入均为当前时间步的小批量输入$X_t \in \mathbb{R}^{n \times d}$和上一时间步的隐藏状态$H_{t-1} \in \mathbb{R}^{n \times h}$，输出由激活函数为sigmoid函数的全连接层得到:

$$
R_t = \sigma (X_t W_{xr} + H_{t-1} W_{hr} + \vec{b}_r) \in \mathbb{R}^{n \times h}
$$
$$
Z_t = \sigma (X_t W_{xz} + H_{t-1} W_{hz} + \vec{b}_z) \in \mathbb{R}^{n \times h},
$$

其中$W_{xr},W_{xz} \in \mathbb{R}^{d \times h}$和$W_{hr},W_{hz} \in \mathbb{R}^{h \times h}$为权重参数，$\vec{b}_r,\vec{b}_z \in \mathbb{R}^{1 \times h}$为偏置。选择sigmoid作为激活函数是为了将这两个逻辑门的输出限定在0到1之间。

随后，将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法。如果重置门中元素值接近0，那么意味着重置对应隐藏状态元素为0，即丢弃上一时间步的隐藏状态。如果元素值接近1，那么表示保留上一时间步的隐藏状态。然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数tanh的全连接层计算出候选隐藏状态，其所有元素的值域为$[−1,1]$。

![avatar](../resource/gru_2.svg)

候选隐藏状态的计算表达式为

$$
\tilde{H}_t = tanh (X_t W_{xh} + (H_{t-1} \odot R_t) W_{hh} + \vec{b}_h) \in \mathbb{R}^{n \times h},
$$

其中$W_{xh} \in \mathbb{R}^{d \times h},W_{hh} \in \mathbb{R}^{h \times h}$为权重参数，$\vec{b}_h$为偏置。**重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态，从而更好地捕捉时间序列里短期的依赖关系**。而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。

然后，计算当前时间步的隐藏状态$H_t \in \mathbb{R}^{n \times h}$：

$$
H_t = Z_t \odot H_{t-1} + (1 - Z_t) \odot \tilde{H}_t
$$

![avatar](../resource/gru_3.svg)

更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新。假设更新门在时间步$t'$到$t$（$t' < t$）之间一直近似1，那么在时间步$t'$到$t$之间的输入信息几乎没有流入时间步$t$的隐藏状态$H_t$。这种现象可以理解为：**较早时刻的隐藏状态$H_{t'-1}$一直通过时间保存并传递至当前的时间步$t$**。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

最后，时间步$t$的输出的计算方式不变，仍为

$$
O_t = HW_{hq} + b_q \in \mathbb{R}^{n \times q}.
$$

### 2 从零开始实现

In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import my_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

corpus_indices, char_to_idx, idx_to_char, vocab_size = my_utils.load_data_jay_lyrics()

初始化模型参数：

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()
    W_xr, W_hr, b_r = _three()
    W_xh, W_hh, b_h = _three()
    
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

In [3]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),)

定义GRU模型：

In [4]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilde = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilde
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

训练：

In [5]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [6]:
my_utils.rnn_train_and_predict(gru, get_params, init_gru_state, num_hiddens,
                               vocab_size, device, corpus_indices, idx_to_char,
                               char_to_idx, False, num_epochs, num_steps,
                               lr, clipping_theta, batch_size, pred_period,
                               pred_len, prefixes)

epoch 40, perplexity 151.188831, time 0.97 sec
 - 分开 我想你的让我不想想想想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想
 - 不分开 我想你的让我不想想想想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想
epoch 80, perplexity 33.344767, time 0.99 sec
 - 分开 一直我不多 你爱我不多 你爱我不多 你爱我不多 你爱我不多 你爱我不多 你爱我不多 你爱我不多 你
 - 不分开 不知不觉 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不
epoch 120, perplexity 6.215947, time 1.00 sec
 - 分开 一直我 你子 是对我 说发 你想很久 难着我 说你怎么面我的家不开 我给你的爱不美西 坏坏的让我疯
 - 不分开  没有一直人慢 我想想这样牵着你 想要和你斯坦堡 一只看对落落在一直 我想能这样 你已 这样的黑蜜
epoch 160, perplexity 1.919414, time 0.95 sec
 - 分开 这样的话笑笑 帅呆了我 全场盯人防守 篮下禁区游走 快攻抢篮板球 得分都靠我 你拿着球不投 又不会
 - 不分开  没有回离开人开开 不懂再到我满惯 誓发现迷了泪被 双截棍柔了带刚 想要去河南嵩山 学少林跟武当 


### 3 简洁实现

In [7]:
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = my_utils.RNNModel(gru_layer, vocab_size).to(device)
my_utils.rnn_train_and_predict_torch(model, num_hiddens, vocab_size, device, 
                            corpus_indices, idx_to_char, char_to_idx, 
                            num_epochs, num_steps, lr, clipping_theta, 
                            batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.016933, time 0.71 sec
 - 分开始想要 我的认真败给黑色幽默 走过了很多地方 我来到伊斯坦堡 就像是童话故事  有教堂有城堡 每天忙
 - 不分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
epoch 80, perplexity 1.022492, time 0.71 sec
 - 分开 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要再
 - 不分开始 担心今天的你过得好不好 整个画面是你 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道
epoch 120, perplexity 1.010579, time 0.74 sec
 - 分开 我不懂 说了没用 他的笑容 有何不同 在你心中 我不再受宠 我的天空 是雨是风 还是彩虹 你在操纵
 - 不分开  你叫我怎么跟你像 不要再这样打我妈妈 我说的话 你甘会听 不要再这样打我妈妈 难道你手不会痛吗 
epoch 160, perplexity 1.555129, time 0.73 sec
 - 分开始 爱像 让它喘不过气 已无能为力的让我感动 穿梭时间的画面  一壶好酒 再来一碗热粥 配上几斤的牛
 - 不分开 它在休 语沉默 娘子依旧每日折一枝杨柳 你爸的太快就走 它在许愿池里轻轻叹息 太多 没有 一壶好酒
